File: 
- tree_structure.py là cấu trúc cây     
- tree_builder.py là xây dựng cây
- tree_retriever.py là trích xuất câu trả lời từ cây
sử dụng truy vấn: retrieve_information_collapse_tree



Tôi sẽ phân tích code của bạn:

1. **Model QA và Embedding đang sử dụng:**
- QA Model: Mặc định sử dụng GPT3TurboQAModel (gpt-3.5-turbo)

```63:112:raptor/QAModels.py
class GPT3TurboQAModel(BaseQAModel):
    def __init__(self, model="gpt-3.5-turbo"):
        """
        Initializes the GPT-3 model with the specified model version.

        Args:
            model (str, optional): The GPT-3 model version to use for generating summaries. Defaults to "text-davinci-003".
        """
        self.model = model
        self.client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

    @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
    def _attempt_answer_question(
        self, context, question, max_tokens=150, stop_sequence=None
    ):
        """
        Generates a summary of the given context using the GPT-3 model.

        Args:
            context (str): The text to summarize.
            max_tokens (int, optional): The maximum number of tokens in the generated summary. Defaults to 150.
            stop_sequence (str, optional): The sequence at which to stop summarization. Defaults to None.

        Returns:
            str: The generated summary.
        """
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are Question Answering Portal"},
                {
                    "role": "user",
                    "content": f"Given Context: {context} Give the best full answer amongst the option to question {question}",
                },
            ],
            temperature=0,
        )

        return response.choices[0].message.content.strip()
    @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
    def answer_question(self, context, question, max_tokens=150, stop_sequence=None):

        try:
            return self._attempt_answer_question(
                context, question, max_tokens=max_tokens, stop_sequence=stop_sequence
            )
        except Exception as e:
            print(e)
            return e
```


- Embedding Model: Mặc định sử dụng OpenAIEmbeddingModel (text-embedding-ada-002)

```25:40:raptor/EmbeddingModels.py
        return (
            self.client.embeddings.create(input=[text], model=self.model)
            .data[0]
            .embedding
        )


class SBertEmbeddingModel(BaseEmbeddingModel):
    def __init__(self, model_name="sentence-transformers/multi-qa-mpnet-base-cos-v1"):
        self.model = SentenceTransformer(model_name)

    def create_embedding(self, text):
        return self.model.encode(text)
```


2. **Cách tạo cây:**
- Sử dụng ClusterTreeBuilder (tree_builder_type="cluster" mặc định)

```55:93:raptor/cluster_tree_builder.py
    def construct_tree(
        self,
        current_level_nodes: Dict[int, Node],
        all_tree_nodes: Dict[int, Node],
        layer_to_nodes: Dict[int, List[Node]],
        use_multithreading: bool = False,
    ) -> Dict[int, Node]:
        logging.info("Using Cluster TreeBuilder")

        next_node_index = len(all_tree_nodes)

        def process_cluster(
            cluster, new_level_nodes, next_node_index, summarization_length, lock
        ):
            node_texts = get_text(cluster)

            summarized_text = self.summarize(
                context=node_texts,
                max_tokens=summarization_length,
            )

            logging.info(
                f"Node Texts Length: {len(self.tokenizer.encode(node_texts))}, Summarized Text Length: {len(self.tokenizer.encode(summarized_text))}"
            )

            __, new_parent_node = self.create_node(
                next_node_index, summarized_text, {node.index for node in cluster}
            )

            with lock:
                new_level_nodes[next_node_index] = new_parent_node

        for layer in range(self.num_layers):

            new_level_nodes = {}

            logging.info(f"Constructing Layer {layer}")

            node_list_current_layer = get_node_list(current_level_nodes)
```


Quy trình:
- Đọc văn bản từ file
- Chia văn bản thành các node
- Tạo embedding cho mỗi node
- Gom cụm (cluster) các node dựa trên độ tương đồng embedding
- Tạo node cha bằng cách tóm tắt nội dung của cụm node con

3. **Builder Type:**
- Đang sử dụng kiểu "cluster" (ClusterTreeBuilder)
- Cấu hình trong RetrievalAugmentationConfig:

```89:105:raptor/RetrievalAugmentation.py
        # Set TreeBuilderConfig
        tree_builder_class, tree_builder_config_class = supported_tree_builders[
            tree_builder_type
        ]
        if tree_builder_config is None:
            tree_builder_config = tree_builder_config_class(
                tokenizer=tb_tokenizer,
                max_tokens=tb_max_tokens,
                num_layers=tb_num_layers,
                threshold=tb_threshold,
                top_k=tb_top_k,
                selection_mode=tb_selection_mode,
                summarization_length=tb_summarization_length,
                summarization_model=tb_summarization_model,
                embedding_models=tb_embedding_models,
                cluster_embedding_model=tb_cluster_embedding_model,
            )
```


4. **Hàm Summary Node:**
- Đang hoạt động và sử dụng GPT3TurboSummarizationModel mặc định

```17:44:raptor/SummarizationModels.py
class GPT3TurboSummarizationModel(BaseSummarizationModel):
    def __init__(self, model="gpt-3.5-turbo"):

        self.model = model

    @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
    def summarize(self, context, max_tokens=500, stop_sequence=None):

        try:
            client = OpenAI()

            response = client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {
                        "role": "user",
                        "content": f"Write a summary of the following, including as many key details as possible: {context}:",
                    },
                ],
                max_tokens=max_tokens,
            )

            return response.choices[0].message.content

        except Exception as e:
            print(e)
            return e
```


Quy trình summarize:
- Lấy text từ các node con
- Gửi đến GPT-3.5-turbo để tóm tắt
- Tạo node mới với nội dung đã tóm tắt

5. **Cấu trúc hoạt động tổng thể:**
- Đọc file -> Tạo nodes
- Build cây theo cluster
- Lưu cây
- Khi query:
  - Tìm nodes liên quan bằng embedding
  - Trích xuất context từ nodes
  - Dùng QA model để trả lời từ context

Bạn có thể tùy chỉnh các model bằng cách truyền vào RetrievalAugmentationConfig khi khởi tạo.

# 2. Cách thức hàm Summary 



Hàm `construct_tree` trong `tree_builder.py` là một phương thức trừu tượng (abstract method) được sử dụng để xây dựng cấu trúc cây phân cấp cho văn bản. Đây là phân tích chi tiết:

1. **Mục đích:**
- Xây dựng cấu trúc cây phân cấp từ các node lá (leaf nodes)
- Tạo các tầng (layer) của cây bằng cách tóm tắt các node liên quan
- Quản lý mối quan hệ giữa các node cha-con

2. **Tham số đầu vào:**

```298:304:raptor/tree_builder.py
    def construct_tree(
        self,
        current_level_nodes: Dict[int, Node],
        all_tree_nodes: Dict[int, Node],
        layer_to_nodes: Dict[int, List[Node]],
        use_multithreading: bool = True,
    ) -> Dict[int, Node]:
```

- `current_level_nodes`: Dictionary chứa các node ở tầng hiện tại
- `all_tree_nodes`: Dictionary chứa tất cả các node trong cây
- `layer_to_nodes`: Dictionary ánh xạ từ số tầng đến danh sách node trong tầng đó
- `use_multithreading`: Cho phép xử lý đa luồng

3. **Cách hoạt động:**
Có 2 cách triển khai chính:

a) **Cluster-based (ClusterTreeBuilder):**

```55:107:raptor/cluster_tree_builder.py
    def construct_tree(
        self,
        current_level_nodes: Dict[int, Node],
        all_tree_nodes: Dict[int, Node],
        layer_to_nodes: Dict[int, List[Node]],
        use_multithreading: bool = False,
    ) -> Dict[int, Node]:
        logging.info("Using Cluster TreeBuilder")

        next_node_index = len(all_tree_nodes)

        def process_cluster(
            cluster, new_level_nodes, next_node_index, summarization_length, lock
        ):
            node_texts = get_text(cluster)

            summarized_text = self.summarize(
                context=node_texts,
                max_tokens=summarization_length,
            )

            logging.info(
                f"Node Texts Length: {len(self.tokenizer.encode(node_texts))}, Summarized Text Length: {len(self.tokenizer.encode(summarized_text))}"
            )

            __, new_parent_node = self.create_node(
                next_node_index, summarized_text, {node.index for node in cluster}
            )

            with lock:
                new_level_nodes[next_node_index] = new_parent_node

        for layer in range(self.num_layers):

            new_level_nodes = {}

            logging.info(f"Constructing Layer {layer}")

            node_list_current_layer = get_node_list(current_level_nodes)
            if len(node_list_current_layer) <= self.reduction_dimension + 1:
                self.num_layers = layer
                logging.info(
                    f"Stopping Layer construction: Cannot Create More Layers. Total Layers in tree: {layer}"
                )
                break

            clusters = self.clustering_algorithm.perform_clustering(
                node_list_current_layer,
                self.cluster_embedding_model,
                reduction_dimension=self.reduction_dimension,
                **self.clustering_params,
            )
```

- Gom nhóm các node theo độ tương đồng embedding
- Tóm tắt nội dung của mỗi cụm để tạo node cha
- Xây dựng cây từ dưới lên theo từng tầng

b) **Transformer-like (đã bị comment out):**

```319:369:raptor/tree_builder.py
        # logging.info("Using Transformer-like TreeBuilder")

        # def process_node(idx, current_level_nodes, new_level_nodes, all_tree_nodes, next_node_index, lock):
        #     relevant_nodes_chunk = self.get_relevant_nodes(
        #         current_level_nodes[idx], current_level_nodes
        #     )

        #     node_texts = get_text(relevant_nodes_chunk)

        #     summarized_text = self.summarize(
        #         context=node_texts,
        #         max_tokens=self.summarization_length,
        #     )

        #     logging.info(
        #         f"Node Texts Length: {len(self.tokenizer.encode(node_texts))}, Summarized Text Length: {len(self.tokenizer.encode(summarized_text))}"
        #     )

        #     next_node_index, new_parent_node = self.create_node(
        #         next_node_index,
        #         summarized_text,
        #         {node.index for node in relevant_nodes_chunk}
        #     )

        #     with lock:
        #         new_level_nodes[next_node_index] = new_parent_node

        # for layer in range(self.num_layers):
        #     logging.info(f"Constructing Layer {layer}: ")

        #     node_list_current_layer = get_node_list(current_level_nodes)
        #     next_node_index = len(all_tree_nodes)

        #     new_level_nodes = {}
        #     lock = Lock()

        #     if use_multithreading:
        #         with ThreadPoolExecutor() as executor:
        #             for idx in range(0, len(node_list_current_layer)):
        #                 executor.submit(process_node, idx, node_list_current_layer, new_level_nodes, all_tree_nodes, next_node_index, lock)
        #                 next_node_index += 1
        #             executor.shutdown(wait=True)
        #     else:
        #         for idx in range(0, len(node_list_current_layer)):
        #             process_node(idx, node_list_current_layer, new_level_nodes, all_tree_nodes, next_node_index, lock)

        #     layer_to_nodes[layer + 1] = list(new_level_nodes.values())
        #     current_level_nodes = new_level_nodes
        #     all_tree_nodes.update(new_level_nodes)

        # return new_level_nodes
```

- Tìm các node liên quan dựa trên khoảng cách embedding
- Tóm tắt nội dung của các node liên quan
- Xây dựng cây theo kiểu attention mechanism

4. **Kết quả:**
- Trả về Dictionary chứa các node gốc (root nodes)
- Cập nhật `all_tree_nodes` và `layer_to_nodes` trong quá trình xây dựng

Hàm này là một phần quan trọng trong việc tổ chức và tóm tắt thông tin theo cấu trúc phân cấp, giúp cho việc truy xuất thông tin hiệu quả hơn.

# test summary node 



Tôi sẽ giải thích vấn đề một cách đơn giản:

1. **Cấu trúc cây hiện tại:**
- Có 2 cách xây dựng cây:
  1. Transformer-like (đã bị comment out trong `tree_builder.py`)
  2. Cluster-based (đang sử dụng trong `cluster_tree_builder.py`)

2. **Vấn đề với Cluster-based:**

```93:100:raptor/cluster_tree_builder.py
            node_list_current_layer = get_node_list(current_level_nodes)
            if len(node_list_current_layer) <= self.reduction_dimension + 1:
                self.num_layers = layer
                logging.info(
                    f"Stopping Layer construction: Cannot Create More Layers. Total Layers in tree: {layer}"
                )
                break
```


- Cây dừng lại ở layer 0 vì điều kiện:
  - `reduction_dimension = 10` (mặc định)
  - Số node hiện tại = 10
  - Khi `len(nodes) <= reduction_dimension + 1` thì dừng
  - Do đó: `10 <= 10 + 1` thỏa mãn ngay từ đầu

3. **Giải pháp:**
- Giảm `reduction_dimension` xuống (ví dụ: 3) để cho phép tạo nhiều layer hơn:

```python
config = RetrievalAugmentationConfig(
    reduction_dimension=3  # Thay vì 10
)
```

4. **Quy trình xây dựng cây đúng:**
- Layer 0: 10 nodes gốc
- Layer 1: Gom 10 nodes thành ~3 cụm, tóm tắt mỗi cụm
- Layer 2: Gom 3 nodes thành 1 cụm, tạo root node

5. **Hàm Summary hoạt động nhưng chưa được gọi:**

```66:85:raptor/cluster_tree_builder.py
        def process_cluster(
            cluster, new_level_nodes, next_node_index, summarization_length, lock
        ):
            node_texts = get_text(cluster)

            summarized_text = self.summarize(
                context=node_texts,
                max_tokens=summarization_length,
            )

            logging.info(
                f"Node Texts Length: {len(self.tokenizer.encode(node_texts))}, Summarized Text Length: {len(self.tokenizer.encode(summarized_text))}"
            )

            __, new_parent_node = self.create_node(
                next_node_index, summarized_text, {node.index for node in cluster}
            )

            with lock:
                new_level_nodes[next_node_index] = new_parent_node
```


- Hàm `process_cluster` có khả năng tóm tắt text
- Nhưng chưa được gọi vì cây dừng quá sớm ở layer 0

Tóm lại: Cần điều chỉnh `reduction_dimension` nhỏ hơn để cây có thể xây dựng nhiều tầng và kích hoạt quá trình tóm tắt.

# 4. Output 

(.venv) (base) PS D:\OneDrive - Hanoi University of Science and Technology\GIT\RAG_RAPTOR>
         python code_testReadme_4omini_v2.py
2024-11-10 13:30:31,435 - Loading faiss with AVX2 support.
2024-11-10 13:30:31,478 - Successfully loaded faiss with AVX2 support.
2024-11-10 13:30:32,344 - Successfully initialized TreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summarization Length: 100
            Summarization Model: <raptor.SummarizationModels.GPT3TurboSummarizationModel object at 0x0000021426BA3B80>
            Embedding Models: {'OpenAI': <raptor.EmbeddingModels.OpenAIEmbeddingModel object at 0x0000021426BA3BB0>}
            Cluster Embedding Model: OpenAI
        
        Reduction Dimension: 3
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}
        
2024-11-10 13:30:32,344 - Successfully initialized ClusterTreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summarization Length: 100
            Summarization Model: <raptor.SummarizationModels.GPT3TurboSummarizationModel object at 0x0000021426BA3B80>
            Embedding Models: {'OpenAI': <raptor.EmbeddingModels.OpenAIEmbeddingModel object at 0x0000021426BA3BB0>}
            Cluster Embedding Model: OpenAI

        Reduction Dimension: 3
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}

2024-11-10 13:30:32,344 - Successfully initialized RetrievalAugmentation with Config
        RetrievalAugmentationConfig:

        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summarization Length: 100
            Summarization Model: <raptor.SummarizationModels.GPT3TurboSummarizationModel object at 0x0000021426BA3B80>
            Embedding Models: {'OpenAI': <raptor.EmbeddingModels.OpenAIEmbeddingModel object at 0x0000021426BA3BB0>}
            Cluster Embedding Model: OpenAI

        Reduction Dimension: 3
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}



        TreeRetrieverConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Context Embedding Model: OpenAI
            Embedding Model: <raptor.EmbeddingModels.OpenAIEmbeddingModel object at 0x00000214D46CE0B0>
            Num Layers: None
            Start Layer: None


            QA Model: <raptor.QAModels.GPT3TurboQAModel object at 0x00000214D46F43D0>
            Tree Builder Type: cluster

2024-11-10 13:30:32,360 - Creating Leaf Nodes
2024-11-10 13:30:34,529 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:30:35,162 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:30:35,222 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:30:35,266 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:30:35,302 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:30:35,363 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:30:35,363 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:30:35,637 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:30:35,744 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:30:35,746 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:30:35,754 - Created 10 Leaf Embeddings
2024-11-10 13:30:35,754 - Building All Nodes
2024-11-10 13:30:35,759 - Using Cluster TreeBuilder
2024-11-10 13:30:35,759 - Constructing Layer 0
D:\OneDrive - Hanoi University of Science and Technology\GIT\RAG_RAPTOR\.venv\lib\site-packages\threadpoolctl.py:1214: RuntimeWarning:
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md        

  warnings.warn(msg, RuntimeWarning)
2024-11-10 13:30:46,609 - Summarization Length: 100
2024-11-10 13:30:48,320 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:30:48,336 - Node Texts Length: 197, Summarized Text Length: 100     
2024-11-10 13:30:48,757 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:30:50,305 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:30:50,305 - Node Texts Length: 89, Summarized Text Length: 100      
2024-11-10 13:30:50,670 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:30:52,150 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:30:52,150 - Node Texts Length: 47, Summarized Text Length: 99       
2024-11-10 13:30:52,704 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:30:54,269 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:30:54,272 - Node Texts Length: 93, Summarized Text Length: 90       
2024-11-10 13:30:54,645 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:30:56,714 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:30:56,716 - Node Texts Length: 200, Summarized Text Length: 77      
2024-11-10 13:30:57,085 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:30:59,636 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:30:59,639 - Node Texts Length: 95, Summarized Text Length: 89       
2024-11-10 13:30:59,960 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:01,092 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:01,108 - Node Texts Length: 95, Summarized Text Length: 61       
2024-11-10 13:31:01,523 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:03,153 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:03,153 - Node Texts Length: 95, Summarized Text Length: 100      
2024-11-10 13:31:03,513 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:03,514 - Constructing Layer 1
2024-11-10 13:31:05,808 - Summarization Length: 100
2024-11-10 13:31:07,828 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:07,828 - Node Texts Length: 77, Summarized Text Length: 100      
2024-11-10 13:31:08,223 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:09,781 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:09,791 - Node Texts Length: 163, Summarized Text Length: 100     
2024-11-10 13:31:10,130 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:11,591 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:11,591 - Node Texts Length: 89, Summarized Text Length: 92       
2024-11-10 13:31:11,950 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:14,236 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:14,236 - Node Texts Length: 101, Summarized Text Length: 100     
2024-11-10 13:31:14,623 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:16,254 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:16,256 - Node Texts Length: 99, Summarized Text Length: 100      
2024-11-10 13:31:16,639 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:18,169 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:26,930 - Node Texts Length: 202, Summarized Text Length: 100
2024-11-10 13:31:27,346 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:28,870 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:28,873 - Node Texts Length: 100, Summarized Text Length: 100
2024-11-10 13:31:29,269 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:31,084 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:31,088 - Node Texts Length: 94, Summarized Text Length: 91
2024-11-10 13:31:31,842 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:34,962 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:34,966 - Node Texts Length: 92, Summarized Text Length: 98
2024-11-10 13:31:35,406 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:37,111 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:37,115 - Node Texts Length: 101, Summarized Text Length: 100
2024-11-10 13:31:37,519 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:37,521 - Constructing Layer 3
2024-11-10 13:31:48,178 - Summarization Length: 100
2024-11-10 13:31:49,671 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:49,671 - Node Texts Length: 91, Summarized Text Length: 100
2024-11-10 13:31:50,688 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:52,221 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:52,221 - Node Texts Length: 197, Summarized Text Length: 100
2024-11-10 13:31:53,173 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:54,680 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:54,682 - Node Texts Length: 101, Summarized Text Length: 100
2024-11-10 13:31:55,147 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:56,619 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:56,619 - Node Texts Length: 101, Summarized Text Length: 100
2024-11-10 13:31:57,509 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:31:59,243 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:31:59,243 - Node Texts Length: 98, Summarized Text Length: 100
2024-11-10 13:32:00,007 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:32:00,023 - Constructing Layer 4
2024-11-10 13:32:04,427 - Summarization Length: 100
2024-11-10 13:32:05,821 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:32:05,826 - Node Texts Length: 101, Summarized Text Length: 100
2024-11-10 13:32:06,798 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:32:08,584 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:32:08,600 - Node Texts Length: 101, Summarized Text Length: 99
2024-11-10 13:32:08,948 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:32:10,679 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:32:10,679 - Node Texts Length: 202, Summarized Text Length: 100
2024-11-10 13:32:11,079 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:32:12,697 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 13:32:12,713 - Node Texts Length: 102, Summarized Text Length: 100
2024-11-10 13:32:13,093 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:32:13,095 - Successfully initialized TreeRetriever with Config
        TreeRetrieverConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Context Embedding Model: OpenAI
            Embedding Model: <raptor.EmbeddingModels.OpenAIEmbeddingModel object at 0x00000214D46CE0B0>
            Num Layers: None
            Start Layer: None

2024-11-10 13:32:13,096 - Using collapsed_tree
2024-11-10 13:32:13,545 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:32:15,418 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Answer:  In the fairy tale of Cinderella, she reached her happy ending through perseverance, kindness, and the help of a magical white bird. Despite facing mistreatment from her stepmother and stepsisters, Cinderella remained resilient and maintained a kind heart. She planted a hazel twig under a tree and prayed for assistance, which led to the appearance of a white bird that granted her wishes. This magical intervention allowed Cinderella to attend the three-day festival hosted by the king for his son's wedding, where she captured the prince's heart with her grace and beauty. Ultimately, Cinderella's unwavering spirit and the assistance of the white bird helped her overcome adversity and find her happily ever after.
2024-11-10 13:32:15,428 - Tree successfully saved to demo/cinderella
2024-11-10 13:32:15,522 - Successfully initialized TreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summarization Length: 100
            Summarization Model: <raptor.SummarizationModels.GPT3TurboSummarizationModel object at 0x00000214D9638D30>
            Embedding Models: {'OpenAI': <raptor.EmbeddingModels.OpenAIEmbeddingModel object at 0x00000214D97F03D0>}
            Cluster Embedding Model: OpenAI

        Reduction Dimension: 3
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}

2024-11-10 13:32:15,524 - Successfully initialized ClusterTreeBuilder with Config
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summarization Length: 100
            Summarization Model: <raptor.SummarizationModels.GPT3TurboSummarizationModel object at 0x00000214D9638D30>
            Embedding Models: {'OpenAI': <raptor.EmbeddingModels.OpenAIEmbeddingModel object at 0x00000214D97F03D0>}
            Cluster Embedding Model: OpenAI

        Reduction Dimension: 3
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}

2024-11-10 13:32:15,525 - Successfully initialized TreeRetriever with Config
        TreeRetrieverConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Context Embedding Model: OpenAI
            Embedding Model: <raptor.EmbeddingModels.OpenAIEmbeddingModel object at 0x00000214DA6C4040>
            Num Layers: None
            Start Layer: None

2024-11-10 13:32:15,526 - Successfully initialized RetrievalAugmentation with Config
        RetrievalAugmentationConfig:

        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summarization Length: 100
            Summarization Model: <raptor.SummarizationModels.GPT3TurboSummarizationModel object at 0x00000214D9638D30>
            Embedding Models: {'OpenAI': <raptor.EmbeddingModels.OpenAIEmbeddingModel object at 0x00000214D97F03D0>}
            Cluster Embedding Model: OpenAI

        Reduction Dimension: 3
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}



        TreeRetrieverConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Context Embedding Model: OpenAI
            Embedding Model: <raptor.EmbeddingModels.OpenAIEmbeddingModel object at 0x00000214DA6C4040>
            Num Layers: None
            Start Layer: None


            QA Model: <raptor.QAModels.GPT3TurboQAModel object at 0x00000214D9BA7340>
            Tree Builder Type: cluster

================= Level 0 =================
Index: 36, Text: In the fairy tale of Cinderella, the protagonist asks her father for a hazel twig that touches his hat on his way home, which she plants gravely under a tree and prays for help. A white bird appears and grants her wishes. When the king decides to have a three-day festival celebrating his son's wedding, Cinderella's step-sisters receive invitations and ask for her help with their hair, shoes, and buckles. Despite their mistreatment of her, Cinderella eventually receives

Index: 37, Text: The passage is a heartfelt plea directed towards the birds inhabiting a garden, urging them to assist in sorting beneficial items into a pot and discarding harmful items into the compost. The speaker uses imagery of tamed pigeons and turtle-doves to symbolize harmony and teamwork with these birds. The tone of the passage is cooperative and unified, emphasizing the significance of working alongside nature to nurture the garden. The overarching message stresses the value of partnering with the natural world to achieve mutual benefit and sustainability.

Index: 38, Text: The passage provides a look into Cinderella's difficult life, where she faces mistreatment from her stepmother and stepsisters. She is subjected to menial and degrading tasks such as picking peas and lentils from ashes, which leaves her looking neglected and dusty. Cinderella's lack of a proper bed forces her to sleep by the hearth among cinders, earning her the nickname "Cinderella." An upcoming event related to Cinderella's father is hinted at in the narrative, suggesting potential

Index: 39, Text: In the story, a princess faces degradation and mistreatment by her peers who strip her of her royal attire and force her to work in the kitchen, performing menial tasks like fetching water, cooking, doing laundry, and starting fires. Despite her royal status, she is humiliated and subjected to cruelty by her sisters who increase her suffering through torment and abuse. Simultaneously, a wealthy man's wife imparts valuable advice to their daughter before passing away, encouraging her to live virtuously. The

================= Level 1 ================= 
Index: 33, Text: In the fairy tale of Cinderella, the titular character asks her father for the first branch that touches his hat on his way home, which ends up being a hazel twig. When Cinderella prays under a tree, a white bird appears and grants her wishes. The king decides to host a three-day festival to celebrate his son's wedding. Cinderella's step-sisters, excitedly receiving invitations to the event, ask her for help with their hair, shoes, and buckles. Despite

Index: 34, Text: The passage is a heartfelt plea directed towards the birds inhabiting a garden, urging them to assist in separating beneficial items into a pot and discarding harmful items into the compost. The speaker evokes imagery of tamed pigeons and turtle-doves to symbolize a sense of harmony and teamwork with these winged creatures. The tone of the passage is one of cooperation and unity, highlighting the importance of working alongside nature to nurture the garden. The overarching message emphasizes the value of partnering with the natural world

Index: 35, Text: In the excerpt, Cinderella's cruel stepmother compels her to go to a festival even though she lacks suitable attire and appears shabby. To prevent Cinderella from attending, the stepmother gives her an unachievable challenge: to separate lentils from ashes in a restricted timeframe. This task appears intended to thwart Cinderella's aspirations and showcase the stepmother's spiteful character. The grim scene emphasizes the stepmother's lack of compassion and Cinderella's perseverance as she deals with her challenging

Index: 31, Text: The passage offers a glimpse into the life of Cinderella, a young girl who faces mistreatment at the hands of her stepmother and stepsisters. She is made to perform degrading tasks such as picking peas and lentils from ashes, resulting in her looking dusty and neglected. Lacking a proper bed, Cinderella sleeps by the hearth among the cinders, which leads to her being called Cinderella. The narrative hints at an imminent event related to Cinderella's father, suggesting potential future

Index: 32, Text: In the story, a princess is stripped of her beautiful attire and forced to work tirelessly in the kitchen by her peers. Despite her royal lineage, she is humiliated and subjected to cruel treatment, including physical tasks like fetching water, starting fires, cooking meals, and doing laundry. Her sisters torment and abuse her, increasing her suffering.

Meanwhile, a wealthy man's wife imparts valuable words of wisdom to their daughter before passing away, urging her to live virtuously. The daughter faithfully visits her mother

================= Level 2 =================
Index: 26, Text: In the fairy tale, Cinderella asks for the first branch touching his hat on his way home, which turns out to be a hazel twig. When Cinderella prays under a tree, a white bird fulfills her wishes. The king plans a three-day festival for his son's wedding. Cinderella's step-sisters receive invitations and prepare excitedly, asking Cinderella for help with their hair, shoes, and buckles. Despite longing to attend the dance, Cinderella is denied permission

Index: 27, Text: The poem or passage is a heartfelt plea to the birds in a garden, enlisting their help in sorting beneficial items into a pot and discarding harmful items into the compost. The speaker draws upon imagery of tamed pigeons and turtle-doves to depict a sense of harmony and collaboration with these winged creatures. The overall tone of the passage conveys a strong message of cooperation and unity with nature, stressing the significance of partnering with the natural world to care for the garden. Through the speaker's

Index: 29, Text: In the excerpt, Cinderella's cruel stepmother forces her to attend a festival despite her shabby appearance and lack of proper clothing. To hinder Cinderella's chances of going, the stepmother assigns her the impossible task of separating lentils from ashes within a tight time limit. This task seems designed to sabotage Cinderella's hopes and highlight the stepmother's malicious nature. The scene underscores the stepmother's heartlessness and Cinderella's struggle as she copes with her difficult situation.

Index: 28, Text: The passage introduces the character of Cinderella, a young girl mistreated by her stepmother and stepsisters. They assign her degrading tasks like picking peas and lentils from ashes, which make her appear dusty and unkempt. Due to her lack of a bed, she sleeps near the hearth amidst cinders, earning her the name Cinderella. The story hints at an upcoming event involving Cinderella's father, indicating potential developments in the narrative.

Index: 25, Text: In this story, a princess faces a cruel fate after being stripped of her beautiful attire and dressed in shabby clothes. Despite her royal lineage, she is humiliated and forced to work tirelessly in the kitchen by her peers. Throughout the day, the princess endures physically grueling tasks like fetching water, starting fires, cooking meals, and doing laundry. The mistreatment intensifies as her sisters relentlessly torment and abuse her, subjecting her to various forms of cruelty and mockery.

Index: 30, Text: In a poignant story highlighting love, loss, and resilience, a wealthy man's wife falls ill and imparts valuable words of wisdom to her daughter before passing away. She encourages her daughter to lead a virtuous life. The daughter, filled with determination, faithfully visits her mother's grave daily and upholds the values her mother instilled in her. However, following the passing of winter, the man remarries and his new wife brings along two stunningly beautiful but cruel daughters. These step-sisters

================= Level 3 =================
Index: 19, Text: In a fairy tale, a man asks his step-daughters what gifts they want brought back for them. The first daughter wants dresses, the second daughter desires pearls and jewels, while Cinderella asks for the first branch that touches his hat on his way home. When a hazel twig knocks off his hat, he retrieves it. Cinderella visits a tree to pray and weep three times a day, where a white bird fulfills her wishes. The king hosts a three-day festival for his son

Index: 23, Text: In the story, Cinderella's step-sisters receive invitations to the wedding at the king's palace, causing them to eagerly prepare for the event. They ask Cinderella to assist them with combing their hair, brushing their shoes, and fastening their buckles. Cinderella reluctantly complies with their requests, although she longs to attend the dance herself. She seeks permission from her step-mother to accompany her step-sisters to the palace, but her plea is dismissed without consideration, leaving

Index: 22, Text: The poem or passage is a plea for the help of birds in a garden, asking them to assist in the task of sorting good items into a pot and disposing of bad items into the compost. The speaker engages with the image of tamed pigeons and turtle-doves to convey a theme of harmony and partnership with the birds in the garden. There is a tone of cooperation and unity with nature throughout the passage, emphasizing the importance of working together with the natural world in tending to the garden.

Index: 20, Text: In the excerpt, Cinderella's stepmother compels her to attend a festival despite her unkempt appearance and lack of suitable attire. To sabotage Cinderella's chances of going, the stepmother demands that she separate lentils from ashes within a tight timeframe. This appears to be an insurmountable challenge, designed to thwart Cinderella's aspirations. The scene underscores the stepmother's cruelty and Cinderella's plight as she navigates her harsh circumstances.     

Index: 24, Text: The passage recounts the story of a young girl known as Cinderella who faces mistreatment from her stepmother and stepsisters. They subject her to demeaning tasks like picking peas and lentils from ashes, leading to her dusty and unkempt appearance. Without a bed of her own, Cinderella sleeps by the hearth amidst cinders, earning her the name Cinderella. The narrative suggests an upcoming event regarding Cinderella's father, teasing further developments in the story. 

Index: 21, Text: In this story, a princess endures harsh treatment after being stripped of her elegant clothes and made to wear shabby garments. Despite her royal status, she is subjected to ridicule by others, who force her to work strenuously in the kitchen. From dawn till dusk, the princess is burdened with physically demanding tasks such as fetching water, lighting fires, cooking, and washing. Adding to her plight, her sisters continuously torment and abuse her, inflicting various forms of mistreatment and taunting.

Index: 18, Text: In a tale about love, loss, and resilience, a wealthy man's wife becomes ill and imparts words of wisdom to her daughter before her passing, urging her to live a good and virtuous life. The daughter dutifully visits her mother's grave daily and continues to embody these values. However, after the winter season, the man remarries a woman with two strikingly beautiful but unkind daughters. These step-daughters subject the kind step-daughter to mistreatment, labeling her with

================= Level 4 =================
Index: 16, Text: In the story, Cinderella visits a tree three times a day to weep and pray, where a little white bird grants her wishes when she expresses them. Meanwhile, the king decides to host a three-day festival, inviting all the beautiful young girls in the land so his son can choose a bride.

Index: 10, Text: In a fairy tale, a man asks his two step-daughters what gifts they want him to bring back for them. The first daughter requests beautiful dresses, while the second asks for pearls and jewels. When the man asks Cinderella, she simply requests the first branch that knocks against his hat on his way home. As he rides through a green thicket, a hazel twig brushes against him and knocks off his hat. He then breaks off the branch and takes it with him.

Upon returning home

Index: 13, Text: In the story, Cinderella's step-sisters are excited to be invited to the wedding at the king's palace. They ask Cinderella to help them get ready by combing their hair, brushing their shoes, and fastening their buckles. Cinderella reluctantly obeys their requests but feels sad because she also wishes to attend the dance. She asks her step-mother for permission to go with them, but her plea is met with indifference.

Index: 12, Text: The poem or passage appears to be a call for assistance from birds in a garden. The person speaking asks the birds to come and help with the task of picking the good items into a pot and separating the bad items into the compost. The imagery includes references to tamed pigeons and turtle-doves, indicating a sense of harmony and partnership with the birds in the garden. The tone of the passage suggests a sense of cooperation and unity with nature in the process of tending to the garden.

Index: 15, Text: The excerpt describes a scene where Cinderella is encouraged by her stepmother to go to a festival even though she is covered in dust and dirt and lacks appropriate clothes and shoes. The stepmother challenges Cinderella to pick out lentils from ashes within two hours, and if she succeeds, she can attend the festival. The stepmother seems to be setting Cinderella up for failure, as this task seems impossible, given Cinderella's current state.

Index: 11, Text: The passage describes how a young girl, referred to as Cinderella, is mistreated by her stepmother and stepsisters. They would make her do menial tasks, such as picking peas and lentils out of ashes, causing her to look dusty and dirty. She had no bed to sleep in, so she rested by the hearth among cinders. Because of her appearance and living conditions, she was nicknamed Cinderella. Additionally, the text hints at an upcoming event where Cinderella's father

Index: 17, Text: In the story, a princess is stripped of her beautiful clothes and forced to wear an old grey bedgown and wooden shoes. Despite her regal background, she is ridiculed by others, who lead her into the kitchen to work tirelessly from dawn to dusk. The princess is tasked with physically demanding chores such as fetching water, lighting fires, cooking, and washing. In addition to her arduous duties, her sisters continuously taunt and mistreat her, subjecting her to various forms of

Index: 14, Text: Summary: The wife of a wealthy man falls ill and advises her daughter to be good and pious before passing away. The daughter visits her mother's grave daily and remains virtuous. When winter passes, the man remarries a woman with two beautiful but cruel daughters. The step-daughters mistreat the kind step-daughter, calling her names and relegating her to the kitchen.

================= Level 5 =================
Index: 6, Text:  Thrice a day cinderella went and sat beneath it, and wept and prayed, and a little white bird always came on the tree, and if cinderella expressed a wish, the bird threw down to her what she had wished for It happened, however, that the king gave orders for a festival which was to last three days, and to which all the beautiful young girls in the country were invited, in order that his son might choose himself a bride

Index: 4, Text: asked his two step-daughters what he should bring back for them Beautiful dresses, said one, pearls and jewels, said the second And you, cinderella, said he, what will you have   Father break off for me the first branch which knocks against your hat on your way home   So he bought beautiful dresses, pearls and jewels for his two step-daughters, and on his way home, as he was riding through a green thicket, a hazel twig brushed against him and

Index: 5, Text: knocked off his hat   Then he broke off the branch and took it with him   When he reached home he gave his step-daughters the things which they had wished for, and to cinderella he gave the branch from the hazel-bush   Cinderella thanked him, went to her mother's grave and planted the branch on it, and wept so much that the tears fell down on it and watered it   And it grew and became a handsome tree

Index: 7, Text:   When the two step-sisters heard that they too were to appear among the number, they were delighted, called cinderella and said, comb our hair for us, brush our shoes and fasten our buckles, for we are going to the wedding at the king's palace Cinderella obeyed, but wept, because she too would have liked to go with them to the dance, and begged her step-mother to allow her to do so

Index: 9, Text: back-door into the garden, and called, you tame pigeons, you turtle-doves, and all you birds beneath the sky, come and help me to pick      the good into the pot,      the bad into the crop

Index: 8, Text:   You go, cinderella, said she, covered in dust and dirt as you are, and would go to the festival   You have no clothes and shoes, and yet would dance   As, however, cinderella went on asking, the step-mother said at last, I have emptied a dish of lentils into the ashes for you, if you have picked them out again in two hours, you shall go with us   The maiden went through the

e good and pious, and then the good God will always protect you, and I will look down on you from heaven and be near you   Thereupon she closed her eyes and departed   Every day the maiden went out to her mother's grave, and wept, and she remained pious and good   When winter came

Index: 1, Text: the snow spread a white sheet over the grave, and by the time the spring sun had drawn it off again, the man had taken another wife The woman had brought with her into the house two daughters, who were beautiful and fair of face, but vile and black of heart Now began a bad time for the poor step-child   Is the stupid goose to sit in the parlor with us, they said   He who wants to eat bread must earn it   Out with the kitchen-wench
e good and pious, and then the good God will always protect you, and I will look down on you from heaven and be near you   Thereupon she closed her eyes and departed   Every day the maiden went out to her mother's grave, and wept, and she remained pious and good   When winter came

Index: 1, Text: the snow spread a white sheet over the grave, and by the time the spring sun had drawn it off again, the man had taken another wife The woman had brought with her into the house two daughters, who were beautiful and fair of face, but vile and black of heart Now began a bad time for the poor step-child   Is the stupid goose to sit in the parlor with us, they said   He who wants to eat bread must earn it   Out with the kitchen-wench

2024-11-10 13:32:15,559 - Using collapsed_tree
2024-11-10 13:32:15,993 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:32:18,233 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Answer after loading tree:  In the fairy tale of Cinderella, she reached her happy ending through perseverance, kindness, and the help of a magical white bird. Despite facing mistreatment from her stepmother and stepsisters, Cinderella remained resilient and maintained a kind heart. She prayed and wept under a tree, where a white bird appeared and granted her wishes. This magical assistance allowed her to attend the three-day festival hosted by the king for his son's wedding.

e good and pious, and then the good God will always protect you, and I will look down on you from heaven and be near you   Thereupon she closed her eyes and departed   Every day the maiden went out to her mother's grave, and wept, and she remained pious and good   When winter came

Index: 1, Text: the snow spread a white sheet over the grave, and by the time the spring sun had drawn it off again, the man had taken another wife The woman had brought with her into the house two daughters, who were beautiful and fair of face, but vile and black of heart Now began a bad time for the poor step-child   Is the stupid goose to sit in the parlor with us, they said   He who wants to eat bread must earn it   Out with the kitchen-wench

2024-11-10 13:32:15,559 - Using collapsed_tree
2024-11-10 13:32:15,993 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:32:18,233 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Answer after loading tree:  In the fairy tale of Cinderella, she reached her happy ending through perseverance, kindness, and the help of a magical white bird. Despite facing mistreatment from her stepmother and stepsisters, Cinderella remained resilient and maintained a kind heae good and pious, and then the good God will always protect you, and I will look down on you from heaven and be near you   Thereupon she closed her eyes and departed   Every day the maiden went out to her mother's grave, and wept, and she remained pious and good   When winter came

Index: 1, Text: the snow spread a white sheet over the grave, and by the time the spring sun had drawn it off again, the man had taken another wife The woman had brought with her into the house two daughters, who were beautiful and fair of face, but vile and black of heart Now began a bad time for the poor step-child   Is the stupid goose to sit in the parlor with us, they said   He who wants to eat bread must earn it   Out with the kitchen-wench

2024-11-10 13:32:15,559 - Using collapsed_tree
2024-11-10 13:32:15,993 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
sed her eyes and departed   Every day the maiden went out to her mother's grave, and wept, and she remained pious and good   When winter came

Index: 1, Text: the snow spread a white sheet over the grave, and by the time the spring sun had drawn it off again, the man had taken another wife The woman had brought with her into the house two daughters, who were beautiful and fair of face, but vile and black of heart Now began a bad time for the poor step-child   Is the stupid goose to sit in the parlor with us, they said   He who wants to eat bread must earn it   Out with the kitchen-wench
e

Index: 1, Text: the snow spread a white sheet over the grave, and by the time the spring sun had drawn it off again, the man had taken another wife The woman had brought with her into the house two daughters, who were beautiful and fair of face, but vile and black of heart Now began a bad time for the poor step-child   Is the stupid goose to sit in the parlor with us, they said   He who wants to eat bread must earn it   Out with the kitchen-wench
Index: 1, Text: the snow spread a white sheet over the grave, and by the time the spring sun had drawn it off again, the man had taken another wife The woman had brought with her into the house two daughters, who were beautiful and fair of face, but vile and black of heart Now began a bad time for the poor step-child   Is the stupid goose to sit in the parlor with us, they said   He who wants to eat bread must earn it   Out with the kitchen-wench
gan a bad time for the poor step-child   Is the stupid goose to sit in the parlor with us, they said   He who wants to eat bread must earn it   Out with the kitchen-wench
t   Out with the kitchen-wench

2024-11-10 13:32:15,559 - Using collapsed_tree
2024-11-10 13:32:15,993 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-10 13:32:18,233 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
ilient and maintained a kind heart. She prayed and wept under a tree, where a white bird appeared and granted her wishes. This magical assistance allowed her to attend the three-day festival hosted by the king for his son's wedding.

Cinderella's selflessness and unwavering faith ultimately led to her happy ending, where she was able to overcome her hardships and find love and happiness at the royal event. Through her resilience and the assistance of the white bird, Cinderella's story teaches us the power of kindness, perseverance, and believing in the possibility of a better future.
(.venv) (base) PS D:\OneDrive - Hanoi University of Science and Technology\GIT\RAG_RAPTOR>













